In [ ]:
from download import get_latlon, get_osm, get_latlon_from_address
from project import Projection, BoundaryBox
from viz import GeoPlotter, Colormap, plot_nodes
from tiling import TileManager
from viz_2d import plot_images
from pathlib import Path
import numpy as np


解析某个位置的经纬度

In [ ]:
# use latlon directly
latlon = np.array((47.3783433, 8.5487747))
# latlon = np.array((1.2882100868743724, 103.78475189208984))

# get latlon from address
# prior_address = "ETH CAB Zurich"
# latlon = get_latlon(prior_address=prior_address)
print(latlon)

In [ ]:
proj = Projection(*latlon)
center = proj.project(latlon)
print(center)

# get the box
tile_size_meters = 64
bbox = BoundaryBox(center, center) + tile_size_meters

In [ ]:
# Show the query area in an interactive map
plot = GeoPlotter(zoom=16)
plot.points(proj.latlonalt[:2], "red", name="location prior", size=10)
plot.bbox(proj.unproject(bbox), "blue", name="map tile")
plot.fig.show()

In [ ]:
# get osm map in the box
path = Path('../data/sets/osm/' + prior_address.replace(' ', '_') + '.json')
get_osm(proj.unproject(bbox + 10), cache_path=path, overwrite=True)

In [ ]:
# Query OpenStreetMap for this area
pixel_per_meter = 2
tiler = TileManager.from_bbox(proj, bbox, pixel_per_meter, path=path)
canvas = tiler.query(bbox)

In [ ]:
# Show the inputs to the model: raster map
map_viz = Colormap.apply(canvas.raster)
plot_images([map_viz], titles=["OpenStreetMap raster"], cmaps="gray")
plot_nodes(0, canvas.raster[2], fontsize=6, size=10)

In [ ]:
# project the raster map to the interactive map
bbox_latlon = proj.unproject(canvas.bbox)
plot = GeoPlotter(zoom=16.5)
plot.raster(map_viz, bbox_latlon, opacity=0.5)
# plot.raster(likelihood_overlay(prob.numpy().max(-1)), proj.unproject(bbox))
plot.points(proj.latlonalt[:2], "red", name="location prior", size=10)
# plot.points(proj.unproject(canvas.to_xy(uv)), "black", name="argmax", size=10)
plot.bbox(bbox_latlon, "blue", name="map tile")
plot.fig.show()